# MFS based MD Simulation

### A Sound scattering simulator based on the method of fundamental solutions

#### Author:

*Nicholas St. Clair, Physics (Kleckner Lab), University of California, Merced*

#### Contributors:
*Arnold Kim, Applied Mathematics, University of California, Merced*

*Dustin Kleckner, Physics, University of California, Merced*

#### Description:
The following notebook makes use of the method of fundamental solutions to compute the sound scattered from multiple sound hard scattering bodies. It determines the forces resultant from said scattering events, some of which are non-linear interactions brought about by interference of acoustic fields. The force data from the MFS scattering code is then fed into a Runge-Kutta based step integrator to produce a molecular dynamics type simulator.

In [1]:
#Import relevant python libraries
import numpy as np
from scipy.integrate import solve_ivp
import time
import matplotlib.pyplot as plt
#Import MFS master class and state it's location
import mfs as MFS
print('MFS class downloaded at:', MFS.__file__)

MFS class downloaded at: c:\users\nstcl\documents\github\mfs\mfs\__init__.py


## 1. Base differential equation

The following cell defines a function which takes a 1 X (6*Np) array containing current position and velocity info and returns an array of the same shape with info updated according to the current velocity and force data. It also takes particle number and radius as function parameters.

*Unfortunately the integrator only accepts 1D arrays...*

In [2]:
def Binding(t, M, Np, a, N, m, μ, scat):
    #initialize arrays
    Out = np.empty_like(M)
    X = np.empty((Np,3))
    #update output array position data
    for n in range(Np):
        X[n] = [M[n*6],M[n*6 + 1], M[n*6 + 2]]
        Out[n*6] = M[n*6 + 3]
        Out[n*6 + 1] = M[n*6 + 4]
        Out[n*6 + 2] = M[n*6 + 5]
    #feed the class the current position data and use to compute forces
    scat.solve(X)
    Ft = scat.force() + scat.contact()
    F = Ft.flatten()
    #update output array velocity data
    #the second terms involved in the force calculation are stokes drag forces
    for n in range(Np):
        Out[n*6 + 3] = (F[n*3] - (6*np.pi*μ*a*M[n*6 + 3])) / m
        Out[n*6 + 4] = (F[n*3 + 1] - (6*np.pi*μ*a*M[n*6 + 4])) / m
        Out[n*6 + 5] = (F[n*3 + 2] - (6*np.pi*μ*a*M[n*6 + 5])) / m
    return Out.flatten();

## 2. Runge-Kutta based solver

The following cell defines initial conditions, as well as any global parameters necessary to set up the simulation.
It then feeds the initial conditions to the solver to be run for a predetermined amount of time.

In [ ]:
#time range of simulation
t_0 = 0
t_f = 10.0

#relevant simulation parameters
#MFS number (number of points in the source lattice)
N = 252
#quadrature number (number of points in evaluations lattice is 2*Nq^2)
Nq = 8
#wavenumber (1/m)
k = 733
#size parameter
ka = 0.1
#particle radius
a = ka / k
# particle mass density (this may be a low estimate?)
ρp = 11
#particle mass
m = ((4 * np.pi / 3) * a ** 3) * ρp
#dynamic viscosity of air
μ = 1.81e-5
#fluid density
rho = 1.225
#Incident field amplitude
phi_a = 1e-5
#Reference acoustic force
F_0 = np.pi * rho * phi_a**2 * k**2 * a**2
#z level of group (m)
zl = 3*np.pi / (4*k)
#initial particle spacing in lattice
spi = 5*a

#number of rows (this creates an nr X nr grid of particles)
nr = 3
#Initial Particle Positions (currently set to a 2d square lattice with small random perturbations)
if nr == 1:
    #number of particles
    Np = 2
    pos = np.array([(0,0,zl),(spi,0,zl)])
elif nr > 1:
    Np = nr ** 2

    shape = np.empty((nr,nr,3))
    base = (0, 0, zl)
    basen = np.full_like(shape, base)
    for i in range(nr):
        basen[:,i,0] += i*spi
        basen[i,:,1] += i*spi
    pos = basen.reshape(Np,3)
pos[:,0] += 0.25 * a * (np.random.rand(Np) - 0.5)
pos[:,1] += 0.25 * a * (np.random.rand(Np) - 0.5)
#Set initial particle Velocities
vel = np.zeros((Np,3))
#Set initial condition array (accepted by the integrator)
ICs = np.empty((Np,6))
ICs[:, :3] = pos
ICs[:, 3:] = vel

#call MFS scatter class with relevant parameters
scat = MFS.Scatter(k=k, a=a, N=N, Nq=Nq, phi_a=phi_a, lattice_type='icos', rho=1.225, source_depth=0.5)
#initiate incident counterpropogating fields in z
scat.incoming_planewaves([1, 1j], [(0, 0, 1), (0, 0, -1)])
#run the simulation
ti = time.time()
nsol = solve_ivp(lambda t,M: Binding(t, M, Np, a, N, m, μ, scat), [t_0,t_f], ICs.flatten(), max_step = 1.0)
tf = time.time()

## 3. Particle Trajectories

The following cell plots the initial and final positions of the acoustically interacting particles, as well as the trajectories taken in between these locations.

In [ ]:
fig,ax = plt.subplots()
for i in range(Np):
    plt.plot(nsol.y[6*i],nsol.y[(6*i)+1],'-.')
    c1 = plt.Circle((nsol.y[6*i,0],nsol.y[(6*i)+1,0]), a, color = 'blue', alpha = 0.5)
    c2 = plt.Circle((nsol.y[6*i,-1],nsol.y[(6*i)+1,-1]), a, color = 'white', ec = 'k')
    ax.add_artist(c1)
    ax.add_artist(c2)

x_bounds = ax.get_xlim()
y_bounds = ax.get_ylim()
ax.set_xlim(x_bounds[0] - a, x_bounds[1] + a)
ax.set_ylim(y_bounds[0] - a, y_bounds[1] + a)
plt.gca().set_aspect(1)
plt.xlabel('x')
plt.ylabel('y')
plt.title(f'ka = {ka}')
plt.tight_layout()
plt.show()

## 4. Particle Velocities

The following cell plots the mean particle velocity as a function of time.

In [ ]:
plt.plot(nsol.t, (np.sqrt(nsol.y[3]**2 + nsol.y[4]**2) + np.sqrt(nsol.y[9]**2 + nsol.y[10]**2) \
        + np.sqrt(nsol.y[15]**2 + nsol.y[16]**2) + np.sqrt(nsol.y[21]**2 + nsol.y[22]**2) \
        + np.sqrt(nsol.y[27]**2 + nsol.y[28]**2) + np.sqrt(nsol.y[33]**2 + nsol.y[34]**2) \
        +np.sqrt(nsol.y[39]**2 + nsol.y[40]**2) + np.sqrt(nsol.y[45]**2 + nsol.y[46]**2)\
        + np.sqrt(nsol.y[51]**2 + nsol.y[52]**2))/Np)
plt.grid()
plt.title('Mean Particle Velocity')
plt.xlabel('Time (s)')
plt.ylabel('V (m/s)')
plt.show()

## 5. Configuration Energy

The following cell determines the acoustic potential energy associated with the configuration at each time step of the simulation. The energy data is then plotted vs time.

In [ ]:
#Initiate empty arrays to store energy and position data
E_t = np.empty(len(nsol.t))
X = np.empty((Np,3))

for i in range(len(nsol.t)):
    
    #take position data from simulation output
    for n in range(Np):
        
        X[n] = (nsol.y[n*6,i],nsol.y[n*6 + 1, i],nsol.y[n*6 + 2,i])
        
    #solve for acoustic potential according to particle locations at timestep i
    scat.solve(X)
    E_N = scat.energy()
    
    #sum the energy over all of the particles in the simulation
    E_t[i] = E_N.sum()

In [ ]:
t_space = np.linspace(0,len(nsol.t),len(nsol.t))
plt.plot(nsol.t, E_t)
plt.grid()
plt.title('Acoustic Potential vs time')
plt.xlabel('t (s)')
plt.ylabel('E (AU)')
plt.show()

## 6. Field visualization

The following cell plots a heatmap of the second order pressure field of the configuration at the final time step of the simulation.

In [ ]:
if __name__ == "__main__":
    #set relevant parameters for field visualization
    #mass density of air
    ρ = 1.225
    #sound speed in air
    c = 343
    #angular frequency of incident sound field
    ω = k * c
    #compressibility factor of incident sound field
    κ = 1 / (ρ * c**2)
    
    #call the scatter class
    scat = MFS.Scatter(N=492, k=k, a=a, phi_a = phi_a, rho=ρ, source_depth=0.5)
    #initiate counterpropagating plane waves in z
    scat.incoming_planewaves([1, 1j], [(0, 0, 1), (0, 0, -1)])
    
    #take final configuration data as particle positions
    pos = np.empty((Np,3))
    
    for i in range(Np):
        pos[i] = nsol.y[i*6:(i*6)+3,-1]
    
    #create a meshgrid over which to plot the second order pressure field
    xpoints = np.linspace(np.amin(pos[:,0])-2*a,np.amax(pos[:,0])+2*a,Ne)
    ypoints = np.linspace(np.amin(pos[:,1])-2*a,np.amax(pos[:,1])+2*a,Ne)
    Ne = 100
    X = np.zeros((Ne,Ne,3))
    
    X[:,:,0] += xpoints.reshape(1,-1)
    X[:,:,1] += ypoints.reshape(-1,1)
    
    #solve for the velocity and velocity potential fields according to particle locations
    scat.solve(pos)
    phi, v = scat.fields(X)
    
    #compute the second order pressure field from velocity and velocity potential
    field = 0.25 * ρ * (k**2 * np.abs(phi)**2 - dot(v, v.conjugate()).real)
    #determine the extent of the window over which to visualize the relevant field
    extent = [np.amin(pos[:,0])-2*a,np.amax(pos[:,0])+2*a,np.amin(pos[:,1])-2*a,np.amax(pos[:,1])+2*a]
    
    #plot the relevant field
    plt.imshow(field, origin='lower',clim = (-np.amax(np.abs(field))/20000,np.amax(np.abs(field))/20000), extent = extent, cmap='RdBu')
    plt.colorbar()
    
    #plot particle locations for clarity
    for X in scat.X:
        plt.gca().add_artist(plt.Circle((X[0], X[1]), scat.a, color='w', ec='k'))

    plt.show()